# Homework III

Diogo Correia (ist199211) & Tomás Esteves (ist199341)

## I. Pen and Paper [11v]

**Given the bivariate observations** $\left\{
\begin{pmatrix}
    1 \\
    2
\end{pmatrix}
,
\begin{pmatrix}
    -1 \\
    1
\end{pmatrix}
,
\begin{pmatrix}
    1 \\
    0
\end{pmatrix}
\right\}
$ **and the multivariate Gaussian mixture**
$$

u_1 = \begin{pmatrix}
    2 \\
    2
\end{pmatrix}
,

u_2 = \begin{pmatrix}
    0 \\
    0
\end{pmatrix}
, 

\Sigma_1 = \begin{pmatrix}
    2 & 1 \\
    1 & 2
\end{pmatrix}
,

\Sigma_2 = \begin{pmatrix}
    2 & 0 \\
    0 & 2
\end{pmatrix}
,

\pi_1 = 0.5,
\pi_2 = 0.5
$$

### 1) [7v] **Perform one epoch of the EM clustering algorithm and determine the new parameters. Indicate all calculus step by step (you can use a computer, however disclose intermediary steps).**

>TODO

### 2) Given the updated parameters computed in previous question:

#### a. [1.5v] **perform a hard assignment of observations to clusters under a MAP assumption.**

>TODO

#### b. [2.5v] **compute the silhouette of the larger cluster using the Euclidean distance.**

>TODO

## Programming and Critical Anlaysis[9v]

**Recall the pd_speech.arff dataset from earlier homeworks, centered on the Parkinson diagnosis from speech features. For the following exercises, normalize the data using sklearn’s MinMaxScaler.**

### 1) [4.5v] **Using sklearn, apply k-means clustering fully unsupervisedly (without targets) on the normalized data with 𝑘 = 3 and three different seeds (using random ϵ {0,1,2}). Assess the silhouette and purity of the produced solutions.**

In [ ]:
from scipy.io.arff import loadarff
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, metrics, cluster, mixture
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA

In [ ]:
data = loadarff("../data/pd_speech.arff")
df = pd.DataFrame(data[0])
df["class"] = df["class"].str.decode("utf-8")
X = df.drop("class", axis=1)
y_true = df["class"]

X.head(2)

In [ ]:
X = MinMaxScaler().fit_transform(X)

X = pd.DataFrame(X, columns=df.columns[:-1])

X.head(2)

In [ ]:
from random import random


kmeans_models = []

for i in range(0, 3):

    # parameterize clustering
    kmeans_algo = cluster.KMeans(n_clusters=3, random_state=i, init="random")

    # learn the model
    kmeans_model = kmeans_algo.fit(X)

    # append the model to the list
    kmeans_models.append(kmeans_model)

kmeans_models

In [ ]:
for i in range(0, 3):

    y_pred = kmeans_models[i].labels_

    # compute silhouette
    print(
        f"{i} Silhouette (euclidean): {metrics.silhouette_score(X, y_pred, metric='euclidean')}"
    )
    # print(f"{i} Silhouette (manhattan): {metrics.silhouette_score(X, y_pred, metric='manhattan')}")

In [ ]:
def purity_score(y_true, y_pred):
    # compute contingency/confusion matrix
    confusion_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(confusion_matrix, axis=0)) / np.sum(confusion_matrix)

In [ ]:
for i in range(0, 3):

    y_pred = kmeans_models[i].labels_

    print("Purity:", purity_score(y_true, y_pred))

### 2) [1.5v] **What is causing the non-determinism?**

>TODO

Something about the random initialization of the centroids. The centroids are initialized randomly, so the results are not deterministic. 

### 3) [1.5v] **Using a scatter plot, visualize side-by-side the labeled data using as labels:** 
- i) the original Parkinson diagnoses, 
- ii) the previously learned 𝑘 = 3 clusters (random= 0). 

**To this end, select the two most informative features as axes and color observations according to their label. For feature selection, select the two input variables with highest variance on the MinMax normalized data.**

In [ ]:
# get the columns with the highest variance
selected_features = list(X.var().sort_values(ascending=False).head(2).index)

selected_features[0]

In [ ]:
colors = ListedColormap(["#FF0000", "#0000FF", "#00FF00"])

# predicted data of the first model
y_pred = kmeans_models[0].labels_

# Original data
figure = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
ax.set_facecolor("#BEBEBE")
ax.scatter(X[selected_features[0]], X[selected_features[1]], c=y_true, cmap=colors)
plt.xlabel(selected_features[0])
plt.ylabel(selected_features[1])

# k = 3 (random = 0)
ax = plt.subplot(1, 2, 2)
ax.scatter(X[selected_features[0]], X[selected_features[1]], c=y_pred, cmap=colors)
plt.xlabel(selected_features[0])
plt.ylabel(selected_features[1])

plt.show()

### 4) [1.5v] **The fraction of variance explained by a principal component is the ratio between the variance of that component (i.e., its eigenvalue) and total variance (i.e., sum of all eigenvalues). How many principal components are necessary to explain more than 80% of variability? Hint: explore the DimReduction notebook to be familiar with PCA in sklearn.**

In [ ]:
pca = PCA(n_components=0.8, svd_solver="full")

pca.fit(X)

print(pca.n_components_)